In [1]:
# model_loader.py
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import os

# Configuration - saves in the same directory as your app.py
MODEL_NAME = "spacemanidol/flan-t5-base-xsum"
LOCAL_MODEL_DIR = "./flan-t5-xsum-model"  # Folder will be created next to app.py
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def setup_model():
    """Download (if needed) and load the model with local caching"""
    # Create model directory if it doesn't exist
    os.makedirs(LOCAL_MODEL_DIR, exist_ok=True)
    
    # Check if model already exists locally
    if not os.path.exists(os.path.join(LOCAL_MODEL_DIR, "pytorch_model.bin")):
        print("Downloading model from Hugging Face Hub...")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
        
        # Save locally for future use
        model.save_pretrained(LOCAL_MODEL_DIR)
        tokenizer.save_pretrained(LOCAL_MODEL_DIR)
        print(f"Model saved to {LOCAL_MODEL_DIR}")
    else:
        print("Loading model from local cache...")
    
    # Load from local files
    tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_DIR)
    model = AutoModelForSeq2SeqLM.from_pretrained(LOCAL_MODEL_DIR).to(DEVICE)
    
    return model, tokenizer

def summarize(text, model, tokenizer, max_length=128):
    """Generate a summary for the input text"""
    inputs = tokenizer(
        "summarize: " + text,
        max_length=1024,
        truncation=True,
        return_tensors="pt"
    ).to(DEVICE)
    
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Initialize model (will auto-download on first run)
model, tokenizer = setup_model()

# Example usage (can be imported by app.py)
if __name__ == "__main__":
    test_article = """
    The James Webb Space Telescope has captured stunning new images of the Cartwheel Galaxy,
    revealing never-before-seen details about star formation and black holes. 
    Located 500 million light-years away, the galaxy's unique ring structure resulted from 
    a high-speed collision between two galaxies. Scientists say these observations provide 
    new insights into galactic evolution.
    """
    
    print("Summary:", summarize(test_article, model, tokenizer))

c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model saved to ./flan-t5-xsum-model
Summary: Astronomers say they have uncovered new insights into the galaxy's ring structure and black holes.
